# Visual Steganography Analysis

This notebook provides highly visual, interactive analysis for cover vs stego images.
- Side-by-side images and amplified differences
- Grayscale histograms and chi-square LSB statistics
- DCT magnitude and WHT magnitude views
- Quality metrics (MSE, PSNR, SSIM)

Use it to generate publication-ready figures quickly.


In [ ]:
from __future__ import annotations
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().resolve().parents[0]))

from io import BytesIO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from stego.lsb import encode_lsb, decode_lsb
from metrics.analysis import compute_metrics, dct_magnitude, wht2, chi_square_lsb
from crypto.pbkdf2_aesgcm import encrypt_aes_gcm, decrypt_aes_gcm, DEFAULT_ITERS

print("✅ Imports ready.")


In [ ]:
# Configure your paths here
COVER_PATH = "../cover.png"  # change as needed
PAYLOAD_PATH = "../secret.txt"  # set to None to skip embedding
PASSWORD = "password123"
USE_ENCRYPT = True
USE_KEYED = True
USE_INVERT = False
OUT_FMT = "PNG"

# Load inputs
cover_img = Image.open(COVER_PATH).convert("RGB")
cover_bytes = pathlib.Path(COVER_PATH).read_bytes()
payload_bytes = None
filename = None
if PAYLOAD_PATH:
    payload_bytes = pathlib.Path(PAYLOAD_PATH).read_bytes()
    filename = pathlib.Path(PAYLOAD_PATH).name

# Optional encryption
blob = payload_bytes
salt = None
iters = None
encrypted_flag = False
if payload_bytes is not None and USE_ENCRYPT:
    blob, salt, iters = encrypt_aes_gcm(payload_bytes, PASSWORD, DEFAULT_ITERS, aad=(filename or "output.bin").encode("utf-8"))
    encrypted_flag = True

# Encode
if payload_bytes is not None:
    stego_bytes = encode_lsb(
        cover_img_bytes=cover_bytes,
        filename=filename or "output.bin",
        payload=blob or b"",
        encrypted=encrypted_flag,
        inverted=USE_INVERT,
        keyed=USE_KEYED,
        out_format=OUT_FMT,
        key_material=(PASSWORD.encode("utf-8") if USE_KEYED else None),
        salt=salt,
        iters=iters,
    )
    stego_img = Image.open(BytesIO(stego_bytes)).convert("RGB")
else:
    stego_img = cover_img

print("✅ Prepared stego image.")


In [ ]:
# Side-by-side and amplified difference
import numpy as np

cover_arr = np.array(cover_img)
stego_arr = np.array(stego_img)
diff = np.abs(stego_arr.astype(np.int16) - cover_arr.astype(np.int16)).astype(np.uint8)
diff_amp = (diff * 16).clip(0, 255).astype(np.uint8)

fig, ax = plt.subplots(1, 3, figsize=(12,4))
ax[0].imshow(cover_img); ax[0].set_title("Cover"); ax[0].axis('off')
ax[1].imshow(stego_img); ax[1].set_title("Stego"); ax[1].axis('off')
ax[2].imshow(diff_amp); ax[2].set_title("Diff ×16"); ax[2].axis('off')
plt.show()


In [ ]:
# Metrics, histograms, and chi-square
from metrics.analysis import compute_metrics, chi_square_lsb

m = compute_metrics(cover_img, stego_img)
print({k: (round(v,4) if isinstance(v, float) else v) for k,v in m.items()})

cov_gray = np.mean(cover_arr, axis=2).astype(np.uint8)
stg_gray = np.mean(stego_arr, axis=2).astype(np.uint8)

cov_hist, _ = np.histogram(cov_gray, bins=256, range=(0, 256))
stg_hist, _ = np.histogram(stg_gray, bins=256, range=(0, 256))

chi_cov = chi_square_lsb(cover_img)
chi_stg = chi_square_lsb(stego_img)
print("Chi-square normalized:", chi_cov['norm_stat'], "→", chi_stg['norm_stat'])

fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].plot(cov_hist, color='gray', alpha=.8, label='Cover')
ax[0].plot(stg_hist, color='tab:blue', alpha=.8, label='Stego')
ax[0].set_title('Grayscale Histogram')
ax[0].legend()
ax[1].bar(['Cover','Stego'], [chi_cov['norm_stat'], chi_stg['norm_stat']], color=['gray','tab:blue'])
ax[1].set_title('Chi-square (normalized)')
plt.show()


In [ ]:
# DCT and WHT magnitude views
from metrics.analysis import dct_magnitude, wht2

dct_cov_g, dct_cov_r, dct_cov_g2, dct_cov_b = dct_magnitude(cover_img)
dct_stg_g, dct_stg_r, dct_stg_g2, dct_stg_b = dct_magnitude(stego_img)

wht_cov, _ = wht2(cover_img)
wht_stg, _ = wht2(stego_img)

fig, ax = plt.subplots(2,3, figsize=(12,6))
ax[0,0].imshow((dct_cov_g*255).astype('uint8'), cmap='inferno'); ax[0,0].set_title('DCT Gray (Cover)'); ax[0,0].axis('off')
ax[0,1].imshow((dct_stg_g*255).astype('uint8'), cmap='inferno'); ax[0,1].set_title('DCT Gray (Stego)'); ax[0,1].axis('off')
ax[0,2].imshow(((dct_stg_g-dct_cov_g).clip(0,1)*255).astype('uint8'), cmap='inferno'); ax[0,2].set_title('DCT Δ (Gray)'); ax[0,2].axis('off')
ax[1,0].imshow((wht_cov*255).astype('uint8'), cmap='magma'); ax[1,0].set_title('WHT (Cover)'); ax[1,0].axis('off')
ax[1,1].imshow((wht_stg*255).astype('uint8'), cmap='magma'); ax[1,1].set_title('WHT (Stego)'); ax[1,1].axis('off')
ax[1,2].imshow((((wht_stg-wht_cov).clip(0,1))*255).astype('uint8'), cmap='magma'); ax[1,2].set_title('WHT Δ'); ax[1,2].axis('off')
plt.tight_layout(); plt.show()
